In [131]:
!pip install keras

# Import Libraries - Pandas, OS, Tensorflow, etc.
import pandas as pd
import os
import shutil 
import os.path
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import numpy as np
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from pathlib import Path


In [132]:
# path, dirs, files = next(os.walk("./data/labeled"))
# file_count = len(files)
# print("Number of files: " + str(file_count))

# train_set = len(files) * 0.6
# valid_set = len(files) * 0.4

# train_count = 0
# validation_count = 0

# overall_count = 0
# for file in os.listdir("./data/labeled"):
#     if (file != ".ipynb_checkpoints"):
#         if (overall_count < train_set):
#             source = './data/labeled/' + str(file)
#             if (file.endswith('0.jpg')):
#                 destination = './train/non-melanoma/' + str(file)
#             else:
#                 destination = './train/melanoma/' + str(file)
                
#             shutil.copy (source, destination)
            
# #             print("Train: " + str(file))
#             train_count += 1
#         else:
#             source1 = './data/labeled/' + str(file)
#             if (file.endswith('0.jpg')):
#                 destination1 = './valid/non-melanoma/' + str(file)
#             else:
#                 destination1 = './valid/melanoma/' + str(file)

#             shutil.copy (source1, destination1)
# #             print("Validation: " + str(file))
#             validation_count += 1
#     overall_count += 1

# print(train_count)
# print(validation_count)
# print(overall_count)

# path, dirs, files = next(os.walk("./valid/melanoma"))
# file_count = len(files)
# print ("Validation dataset length: " + str(file_count))

# path, dirs, files = next(os.walk("./train"))
# file_count = len(files)
# print("Train dataset length: " + str(file_count))

In [133]:
# shutil.rmtree('./train')
# shutil.rmtree('./valid')

# !mkdir train
# !mkdir valid
# !mkdir train/melanoma
# !mkdir train/non-melanoma
# !mkdir valid/melanoma
# !mkdir valid/non-melanoma

In [134]:
# Loading in training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True
)

In [135]:
train_generator = train_datagen.flow_from_directory(
        './train_resize',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

Found 720 images belonging to 2 classes.


In [136]:
# Loading in testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        './valid_resize',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

Found 480 images belonging to 2 classes.


In [137]:
# initialising sequential model and adding layers to it
# cnn = tf.keras.models.Sequential()
# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=(3, 350, 350)))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
# cnn.add(tf.keras.layers.Dense(64, activation='relu'))
# cnn.add(tf.keras.layers.Dense(8, activation='softmax'))

# Initializing sequential model that makes use of convolutional, pooling, and dense layers
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', input_shape=(100, 100, 3), padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = 'same'))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(2, activation='softmax'))

In [138]:
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
# sess = tf.Session(config=config)

In [139]:
cnn.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 100, 100, 5)       140       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 50, 50, 5)         0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 25, 25, 5)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 25, 25, 48)        2208      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 13, 13, 48)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 7, 7, 48)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2352)            

In [140]:
# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, epochs=10)

Epoch 1/10
23/23 [==============================] - 7s 314ms/step - loss: 0.6735 - accuracy: 0.6236 - val_loss: 0.6039 - val_accuracy: 0.7063
Epoch 2/10
23/23 [==============================] - 9s 376ms/step - loss: 0.5997 - accuracy: 0.7014 - val_loss: 0.5482 - val_accuracy: 0.7312
Epoch 3/10
23/23 [==============================] - 5s 208ms/step - loss: 0.5636 - accuracy: 0.7097 - val_loss: 0.5604 - val_accuracy: 0.7292
Epoch 4/10
23/23 [==============================] - 4s 175ms/step - loss: 0.5325 - accuracy: 0.7167 - val_loss: 0.6071 - val_accuracy: 0.7229
Epoch 5/10
23/23 [==============================] - 4s 164ms/step - loss: 0.4980 - accuracy: 0.7639 - val_loss: 0.5216 - val_accuracy: 0.7458
Epoch 6/10
23/23 [==============================] - 5s 212ms/step - loss: 0.4717 - accuracy: 0.7833 - val_loss: 0.5949 - val_accuracy: 0.6833
Epoch 7/10
23/23 [==============================] - 4s 174ms/step - loss: 0.4655 - accuracy: 0.7847 - val_loss: 0.7778 - val_accuracy: 0.6250
Epoch 

In [141]:
# !mkdir model
cnn.save('model/my_model')

INFO:tensorflow:Assets written to: model/my_model/assets


In [143]:
import cv2
import tensorflow as tf
import random

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath) 
    res = cv2.resize(img_array, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
#     return np.reshape(res, (3, IMG_SIZE, IMG_SIZE))
    return res

path, dirs, files = next(os.walk("./data/test"))
file_count = len(files)

predictions = []
prediction_vals = []
final = []
count = 0
l = os.listdir("./data/test")
random.shuffle(l)
for file in l:
    print(file)
    if (file != ".ipynb_checkpoints"):
        image = prepare('data/test/' + str(file)).tolist()
        #print(image.shape)
        prediction = cnn.predict([image])
        prediction_vals.append(prediction)
        if (prediction[0][0] < prediction[0][1]):
            predicted_result = "non-melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"
        else:
            predicted_result = "melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"

        if (predicted_result == final_result):
            count += 1
        predictions.append(predicted_result)
        final.append(final_result)
    
print(count)
print(file_count-1)
    

436_0.jpg
111_1.jpg
446_0.jpg
314_0.jpg
175_1.jpg
231_1.jpg
183_1.jpg
454_0.jpg
212_1.jpg
503_0.jpg
469_0.jpg
350_0.jpg
345_0.jpg
130_1.jpg
275_1.jpg
379_0.jpg
190_1.jpg
419_0.jpg
104_1.jpg
532_0.jpg
473_0.jpg
270_1.jpg
491_0.jpg
462_0.jpg
115_1.jpg
545_0.jpg
155_1.jpg
45_1.jpg
420_0.jpg
51_1.jpg
594_0.jpg
260_1.jpg
266_1.jpg
194_1.jpg
180_1.jpg
82_1.jpg
106_1.jpg
257_1.jpg
330_0.jpg
334_0.jpg
324_0.jpg
121_1.jpg
21_1.jpg
333_0.jpg
11_1.jpg
41_1.jpg
182_1.jpg
415_0.jpg
98_1.jpg
484_0.jpg
188_1.jpg
296_1.jpg
376_0.jpg
244_1.jpg
110_1.jpg
592_0.jpg
438_0.jpg
585_0.jpg
388_0.jpg
529_0.jpg
493_0.jpg
311_0.jpg
539_0.jpg
510_0.jpg
310_0.jpg
302_0.jpg
589_0.jpg
565_0.jpg
167_1.jpg
485_0.jpg
475_0.jpg
230_1.jpg
176_1.jpg
215_1.jpg
60_1.jpg
567_0.jpg
264_1.jpg
135_1.jpg
174_1.jpg
99_1.jpg
323_0.jpg
443_0.jpg
490_0.jpg
33_1.jpg
44_1.jpg
163_1.jpg
384_0.jpg
160_1.jpg
152_1.jpg
72_1.jpg
366_0.jpg
277_1.jpg
173_1.jpg
470_0.jpg
116_1.jpg
392_0.jpg
364_0.jpg
71_1.jpg
287_1.jpg
434_0.jpg
447_0.jpg
331

In [144]:
print(prediction_vals)

[array([[9.9986148e-01, 1.3853319e-04]], dtype=float32), array([[1., 0.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[1.0000000e+00, 1.1630646e-22]], dtype=float32), array([[1., 0.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[6.6270186e-05, 9.9993372e-01]], dtype=float32), array([[7.6568335e-23, 1.0000000e+00]], dtype=float32), array([[0., 1.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[1., 0.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[2.2583546e-37, 1.0000000e+00]], dtype=float32), array([[1.5491513e-35, 1.0000000e+00]], dtype=float32), array([[9.9999988e-01, 1.7273577e-07]], dtype=float32), array([[0., 1.]], dtype=float32), array([[0., 1.]], dtype=float32), array([[1., 0.]], dtype=float

In [71]:
print(prediction)

[[0. 1.]]
